# 基于TimesFM预训练模型的投资组合优化

本教程演示如何使用Google Research的TimesFM预训练模型来进行投资组合优化。TimesFM是一个在大规模金融时间序列数据上进行预训练的模型，可能会带来比传统深度学习模型更好的性能。

## 本教程包括：

1. 数据准备和预处理
2. TimesFM模型设置和配置
3. 模型训练和优化过程
4. 性能评估和可视化分析

In [1]:
# 添加项目根目录到Python路径
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# 导入必要的库
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timesfm
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# 设置随机种子以保证可重复性
torch.manual_seed(42)
np.random.seed(42)

# 设置绘图样式
plt.style.use('default')  # 使用默认样式
sns.set_style("whitegrid")  # 使用seaborn的网格样式

# 设置中文字体
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['WenQuanYi Zen Hei', 'WenQuanYi Micro Hei']
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

# 检查GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

使用设备: cuda


## 数据准备和预处理

首先，我们需要加载和处理金融数据。我们使用Magnificent 7（AAPL、AMZN、GOOGL、META、MSFT、NVDA、TSLA）的每日收盘价数据，并计算对数回报率用于模型训练。

In [2]:
# 加载原始数据
data_raw = pd.read_parquet('../data/mag7_data_raw.parquet')
print("数据形状:", data_raw.shape)
print("\n前5行数据:")
display(data_raw.head())

# 计算对数回报率
close_prices = data_raw
returns = np.log(close_prices / close_prices.shift(1))

# 处理NaN值
returns.iloc[0] = 0  # 第一天的回报率设为0
returns = returns.ffill()  # 用前一个有效值填充其他NaN
returns = returns.bfill()  # 如果数据开头有NaN，用后一个有效值填充

print("\n回报率数据形状:", returns.shape)
print("\n回报率统计描述:")
display(returns.describe())

数据形状: (5027, 35)

前5行数据:


Price          Close                                                   \
Ticker          AAPL    AMZN     GOOGL META       MSFT      NVDA TSLA   
Date                                                                    
2005-09-26  1.616283  2.1670  7.810992  NaN  17.601694  0.251990  NaN   
2005-09-27  1.604275  2.1580  7.802541  NaN  17.650463  0.252831  NaN   
2005-09-28  1.533428  2.1685  7.605204  NaN  17.880327  0.254360  NaN   
2005-09-29  1.571253  2.2395  7.695174  NaN  18.068390  0.259632  NaN   
2005-09-30  1.609379  2.2650  7.865171  NaN  17.922119  0.261924  NaN   

Price           High                    ...       Open                 \
Ticker          AAPL    AMZN     GOOGL  ...       MSFT      NVDA TSLA   
Date                                    ...                             
2005-09-26  1.637897  2.1710  7.976766  ...  17.692244  0.251838  NaN   
2005-09-27  1.628291  2.1865  7.913637  ...  17.671360  0.253060  NaN   
2005-09-28  1.594369  2.1870  7.831371  ...  17.685294  0.253595  NaN   
2005-09-29  1.578758  2.2400  7.722513  ...  17.838530  0.254436  NaN   
2005-09-30  1.610579  2.2920  7.891019  ...  18.047498  0.259784  NaN   

Price           Volume                                                       
Ticker            AAPL       AMZN      GOOGL META      MSFT       NVDA TSLA  
Date                                                                         
2005-09-26   546562800  112328000  395380224  NaN  56203700  406776000  NaN  
2005-09-27   341703600   83470000  274649076  NaN  48797900  404160000  NaN  
2005-09-28  1125544000   64794000  319576104  NaN  71019400  353556000  NaN  
2005-09-29   636846000  127856000  224327448  NaN  66807100  513372000  NaN  
2005-09-30   531633200  121120000  365685948  NaN  57644500  458832000  NaN  

[5 rows x 35 columns]


回报率数据形状: (5027, 35)

回报率统计描述:


Price         Close                                                      \
Ticker         AAPL         AMZN        GOOGL         META         MSFT   
count   5027.000000  5027.000000  5027.000000  5027.000000  5027.000000   
mean       0.000993     0.000929     0.000691     0.000601     0.000669   
std        0.020171     0.023704     0.018871     0.020506     0.017262   
min       -0.197470    -0.246182    -0.123685    -0.306391    -0.159453   
25%       -0.008299    -0.009947    -0.007992    -0.003911    -0.007260   
50%        0.001000     0.000707     0.000790     0.000000     0.000588   
75%        0.011389     0.012336     0.009885     0.005970     0.009117   
max        0.142617     0.238621     0.182251     0.259371     0.170626   

Price                                    High                            ...  \
Ticker         NVDA         TSLA         AAPL         AMZN        GOOGL  ...   
count   5027.000000  5027.000000  5027.000000  5027.000000  5027.000000  ...   
mean       0.001303     0.001107     0.000993     0.000931     0.000688  ...   
std        0.030977     0.031735     0.016896     0.021385     0.016443  ...   
min       -0.367109    -0.236518    -0.133407    -0.163759    -0.094297  ...   
25%       -0.013709    -0.009642    -0.007373    -0.008506    -0.006889  ...   
50%        0.001625     0.000000     0.001028     0.000389     0.000651  ...   
75%        0.016843     0.012933     0.009150     0.009776     0.008323  ...   
max        0.260876     0.218292     0.119419     0.240213     0.175872  ...   

Price          Open                                 Volume               \
Ticker         MSFT         NVDA         TSLA         AAPL         AMZN   
count   5027.000000  5027.000000  5027.000000  5027.000000  5027.000000   
mean       0.000669     0.001301     0.001159    -0.000500    -0.000216   
std        0.016409     0.031687     0.032917     0.315320     0.358852   
min       -0.113241    -0.362972    -0.236893    -1.772503    -1.564264   
25%       -0.007400    -0.014021    -0.010492    -0.201169    -0.213745   
50%        0.000829     0.001411     0.000000    -0.018566    -0.023368   
75%        0.008963     0.017539     0.013172     0.186015     0.196449   
max        0.131589     0.243083     0.305547     1.806734     1.925267   

Price                                                                    
Ticker        GOOGL         META         MSFT         NVDA         TSLA  
count   5027.000000  5027.000000  5027.000000  5027.000000  5027.000000  
mean      -0.000505    -0.000788    -0.000217    -0.000150    -0.000227  
std        0.344308     0.299433     0.328122     0.356322     0.345840  
min       -1.503236    -1.568648    -1.655370    -1.523006    -1.591503  
25%       -0.214204    -0.121572    -0.195340    -0.220339    -0.164003  
50%       -0.019741     0.000000    -0.014108    -0.020866     0.000000  
75%        0.204982     0.079992     0.186077     0.203763     0.122216  
max        1.790403     2.090365     1.978840     1.570406     2.498770  

[8 rows x 35 columns]

In [3]:
# 创建训练数据序列
def create_sequences(data, window_size=20):
    sequences = []
    targets = []
    
    for i in range(len(data) - window_size):
        sequence = data.iloc[i:i+window_size].values
        target = data.iloc[i+window_size].values
        sequences.append(sequence)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

# 创建训练数据
window_size = 20
X, y = create_sequences(returns, window_size)
print("输入数据形状:", X.shape)  # (样本数, 时间步长, 特征数)
print("目标数据形状:", y.shape)  # (样本数, 特征数)

# 将数据转换为PyTorch张量
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

# 分割数据集
train_val_size = int(0.8 * len(X))
train_size = int(0.8 * train_val_size)
X_test = X_tensor[train_val_size:]
y_test = y_tensor[train_val_size:]
X_train = X_tensor[:train_size]
y_train = y_tensor[:train_size]
X_val = X_tensor[train_size:train_val_size]
y_val = y_tensor[train_size:train_val_size]

# 将数据移至GPU（如果可用）
if torch.cuda.is_available():
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    X_val = X_val.cuda()
    y_val = y_val.cuda()
    X_test = X_test.cuda()
    y_test = y_test.cuda()

# 创建数据加载器
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("\n数据集大小:")
print(f"训练集: {len(X_train)} 样本 ({len(X_train)/len(X_tensor):.1%})")
print(f"验证集: {len(X_val)} 样本 ({len(X_val)/len(X_tensor):.1%})")
print(f"测试集: {len(X_test)} 样本 ({len(X_test)/len(X_tensor):.1%})")

输入数据形状: (5007, 20, 35)
目标数据形状: (5007, 35)

数据集大小:
训练集: 3204 样本 (64.0%)
验证集: 801 样本 (16.0%)
测试集: 1002 样本 (20.0%)


In [4]:
#从train_loader中读取一个batch的数据
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)  # (batch_size, window_size, num_features)
    print("Batch y shape:", batch_y.shape)  # (batch_size, num_features)
    break  # 只查看第一个batch

Batch X shape: torch.Size([32, 20, 35])
Batch y shape: torch.Size([32, 35])


## TimesFM模型设置

我们使用Google Research的TimesFM预训练模型作为特征提取器，并在其基础上构建一个投资组合优化模型。模型架构包含以下几个部分：

1. TimesFM预训练模型作为特征提取器
2. 投资组合优化头部网络
3. Softmax层确保权重和为1且非负

In [11]:
# 创建TimesFM模型
timesfm_model = timesfm.TimesFM_2p5_200M_torch()

# 配置并编译模型
print("编译TimesFM模型...")
forecast_config =  timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )

timesfm_model.compile(forecast_config=forecast_config)
print("TimesFM模型编译完成！")

# 创建投资组合优化模型
class TimesFMPortfolioModel(nn.Module):
    def __init__(self, input_size, output_size, timesfm_model: timesfm.TimesFM_2p5_200M_torch, context_len):
        super(TimesFMPortfolioModel, self).__init__()
        self.timesfm = timesfm_model
        self.context_len = context_len
        self.timesfm_model = timesfm_model.model
        # 冻结TimesFM的参数
        for param in self.timesfm.model.parameters():
            param.requires_grad = False
            
        # 投资组合优化头部网络
        # 输入是每个资产的预测收益率，所以维度是 input_size
        self.portfolio_head = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, output_size)
        )
        
        # Softmax层确保权重和为1且非负
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # x的形状: (batch_size, context_len, num_assets)
        batch_size, _, num_assets = x.shape
        
        # 存储每个资产的预测收益
        forecasts = []
        
        # 我们想要预测未来1期的收益
        forecast_horizon = 1
        
        for i in range(num_assets):
            # 获取单个资产的时间序列，形状: (batch_size, context_len)
            asset_series = x[:, :, i]
            asset_series = asset_series.cpu().numpy()
            # 使用forecast进行预测
            # forecast返回 (point_forecast, low_quantile, high_quantile)
            point_forecast, _ = self.timesfm.forecast(
                horizon=forecast_horizon,
                inputs=asset_series
            )
            
            # point_forecast 形状: (batch_size, forecast_horizon)
            # 我们只关心这个预测值
            forecasts.append(point_forecast)
        # convert list of forecasts to a single tensor
        combined_forecasts = torch.stack([torch.tensor(f).squeeze() for f in forecasts], dim=1).to(device)
        # 将所有资产的预测连接起来，形状: (batch_size, num_assets)
        #combined_forecasts = torch.cat(forecasts, dim=1)
        
        # 通过头部网络得到权重
        weights = self.portfolio_head(combined_forecasts)
        
        # 应用Softmax得到最终的投资组合权重
        weights = self.softmax(weights)
        
        return weights

# 初始化模型
input_size = X.shape[2]  # 特征数（股票数量）
output_size = input_size  # 输出权重的维度与股票数量相同
timesfm_portfolio_model = TimesFMPortfolioModel(input_size, output_size, timesfm_model, window_size).to(device)
print(timesfm_portfolio_model)

# 定义投资组合损失函数
def portfolio_loss(weights, returns, risk_aversion=1.0):
    # 计算投资组合收益
    portfolio_return = torch.sum(weights * returns, dim=1)
    
    # 计算平均收益
    expected_return = torch.mean(portfolio_return)
    
    # 计算风险（使用样本标准差的平方而不是方差）
    epsilon = 1e-8
    portfolio_risk = torch.mean((portfolio_return - expected_return) ** 2) + epsilon
    
    # 风险调整后的收益（负号是因为我们要最大化收益，而优化器是最小化损失）
    loss = -(expected_return - risk_aversion * portfolio_risk)
    
    # 添加正则化项以防止权重过于集中
    weight_regularization = torch.mean(torch.sum(weights ** 2, dim=1))
    regularization_factor = 0.01
    loss = loss + regularization_factor * weight_regularization
    
    return loss

# 定义优化器和学习率
optimizer = torch.optim.Adam(timesfm_portfolio_model.parameters(), lr=0.0001)
risk_aversion = 5.0  # 风险厌恶系数

编译TimesFM模型...
TimesFM模型编译完成！
TimesFMPortfolioModel(
  (timesfm_model): TimesFM_2p5_200M_torch_module(
    (tokenizer): ResidualBlock(
      (hidden_layer): Linear(in_features=64, out_features=1280, bias=True)
      (output_layer): Linear(in_features=1280, out_features=1280, bias=True)
      (residual_layer): Linear(in_features=64, out_features=1280, bias=True)
      (activation): SiLU()
    )
    (stacked_xf): ModuleList(
      (0-19): 20 x Transformer(
        (pre_attn_ln): RMSNorm()
        (post_attn_ln): RMSNorm()
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=False)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=False)
          (out): Linear(in_features=1280, out_features=1280, bias=False)
          (query_ln): RMSNorm()
          (key_ln): RMSNorm()
          (rotary_position_embedding): RotaryPositionalEmbedding()
          (pe

In [12]:
# 检查模型参数所在的设备
print("TimesFM Portfolio Model 参数设备分布:")
print("=" * 50)

# 检查模型的各个组件
for name, param in timesfm_portfolio_model.named_parameters():
    print(f"{name}: {param.device}")

print("\n" + "=" * 50)
print(f"模型整体设备: {next(timesfm_portfolio_model.parameters()).device}")

# 检查模型的子模块
print("\n模型子模块设备检查:")
print("-" * 30)
for name, module in timesfm_portfolio_model.named_children():
    if hasattr(module, 'parameters'):
        devices = [p.device for p in module.parameters()]
        print(f"{name}: {devices[0] if devices else 'No parameters'}")
    else:
        print(f"{name}: Non-parameter module")

TimesFM Portfolio Model 参数设备分布:
timesfm_model.tokenizer.hidden_layer.weight: cuda:0
timesfm_model.tokenizer.hidden_layer.bias: cuda:0
timesfm_model.tokenizer.output_layer.weight: cuda:0
timesfm_model.tokenizer.output_layer.bias: cuda:0
timesfm_model.tokenizer.residual_layer.weight: cuda:0
timesfm_model.tokenizer.residual_layer.bias: cuda:0
timesfm_model.stacked_xf.0.pre_attn_ln.scale: cuda:0
timesfm_model.stacked_xf.0.post_attn_ln.scale: cuda:0
timesfm_model.stacked_xf.0.attn.query.weight: cuda:0
timesfm_model.stacked_xf.0.attn.key.weight: cuda:0
timesfm_model.stacked_xf.0.attn.value.weight: cuda:0
timesfm_model.stacked_xf.0.attn.out.weight: cuda:0
timesfm_model.stacked_xf.0.attn.query_ln.scale: cuda:0
timesfm_model.stacked_xf.0.attn.key_ln.scale: cuda:0
timesfm_model.stacked_xf.0.attn.per_dim_scale.per_dim_scale: cuda:0
timesfm_model.stacked_xf.0.pre_ff_ln.scale: cuda:0
timesfm_model.stacked_xf.0.post_ff_ln.scale: cuda:0
timesfm_model.stacked_xf.0.ff0.weight: cuda:0
timesfm_model.stac

## 模型训练和优化

我们现在开始训练模型。训练过程包括：

1. 在训练集上进行梯度下降优化
2. 在验证集上评估模型性能
3. 使用早停机制防止过拟合
4. 保存最佳模型状态

In [ ]:
# 训练模型
num_epochs = 50
train_losses = []
val_losses = []
best_val_loss = float('inf')
best_model_state = None
patience = 15
patience_counter = 0

print(f"开始在 {device} 上训练模型...")

for epoch in range(num_epochs):
    # 训练阶段
    timesfm_portfolio_model.train()
    epoch_train_loss = 0
    
    for batch_X, batch_y in train_loader:
        # 确保数据在正确的设备上
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        weights = timesfm_portfolio_model(batch_X)
        loss = portfolio_loss(weights, batch_y, risk_aversion=risk_aversion)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    epoch_train_loss /= len(train_loader)
    train_losses.append(epoch_train_loss)
    
    # 验证阶段
    timesfm_portfolio_model.eval()
    epoch_val_loss = 0
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            weights = timesfm_portfolio_model(batch_X)
            loss = portfolio_loss(weights, batch_y, risk_aversion=risk_aversion)
            epoch_val_loss += loss.item()
    
    epoch_val_loss /= len(val_loader)
    val_losses.append(epoch_val_loss)
    
    # 早停检查
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model_state = timesfm_portfolio_model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f'早停: 验证损失在 {patience} 个epoch内没有改善')
        break
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {epoch_train_loss:.4f}, '
              f'Val Loss: {epoch_val_loss:.4f}')

# 加载最佳模型状态
if best_model_state is not None:
    timesfm_portfolio_model.load_state_dict(best_model_state)
    print(f'\n已恢复最佳模型（验证损失: {best_val_loss:.4f}）')

开始在 cuda 上训练模型...


## 模型评估和可视化

现在让我们在测试集上评估模型的性能，并可视化以下内容：

1. 训练过程中的损失变化
2. 投资组合权重的动态变化
3. 累积收益曲线

In [ ]:
# 绘制训练和验证损失曲线
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='训练损失')
plt.plot(val_losses, label='验证损失')
plt.xlabel('Epoch')
plt.ylabel('损失')
plt.title('训练和验证损失随时间的变化')
plt.legend()
plt.grid(True)
plt.show()

# 在测试集上评估模型
timesfm_portfolio_model.eval()
test_predictions = []
test_losses = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        weights = timesfm_portfolio_model(batch_X)
        loss = portfolio_loss(weights, batch_y, risk_aversion=risk_aversion)
        test_predictions.append(weights.cpu())
        test_losses.append(loss.item())

# 将预测结果转换为numpy数组
test_predictions = torch.cat(test_predictions, dim=0).numpy()

# 计算测试集上的投资组合表现
test_returns = y_test.cpu().numpy()
portfolio_returns = np.sum(test_predictions * test_returns, axis=1)

# 计算性能指标
mean_return = np.mean(portfolio_returns) * 252  # 年化收益率
std_return = np.std(portfolio_returns) * np.sqrt(252)  # 年化波动率
sharpe_ratio = mean_return / std_return  # 夏普比率

print(f"TimesFM投资组合表现指标:")
print(f"年化收益率: {mean_return:.2%}")
print(f"年化波动率: {std_return:.2%}")
print(f"夏普比率: {sharpe_ratio:.2f}")

# 可视化投资组合权重分配
plt.figure(figsize=(12, 6))
plt.stackplot(range(len(test_predictions)), 
             test_predictions.T,
             labels=returns.columns)
plt.xlabel('时间')
plt.ylabel('权重')
plt.title('TimesFM模型 - 投资组合权重分配随时间的变化')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 绘制累积收益曲线
cumulative_returns = np.cumprod(1 + portfolio_returns) - 1

plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns)
plt.xlabel('时间')
plt.ylabel('累积收益')
plt.title('TimesFM模型 - 投资组合累积收益')
plt.grid(True)
plt.show()

## 结果分析

我们可以通过以下几个方面分析TimesFM模型的性能：

1. 与基准策略（如等权重策略）的对比
2. 投资组合权重的稳定性
3. 交易成本考虑
4. 风险调整后的收益表现

In [ ]:
# 计算等权重策略的表现
equal_weights = np.ones((len(test_returns), len(returns.columns))) / len(returns.columns)
equal_weight_returns = np.sum(equal_weights * test_returns, axis=1)

# 计算等权重策略的指标
ew_mean_return = np.mean(equal_weight_returns) * 252
ew_std_return = np.std(equal_weight_returns) * np.sqrt(252)
ew_sharpe_ratio = ew_mean_return / ew_std_return

# 计算等权重策略的累积收益
ew_cumulative_returns = np.cumprod(1 + equal_weight_returns) - 1

# 比较性能指标
print("性能对比（TimesFM vs 等权重）:")
print("-" * 40)
print(f"{'指标':>15} {'TimesFM':>12} {'等权重':>12}")
print("-" * 40)
print(f"{'年化收益率':>15} {mean_return:>12.2%} {ew_mean_return:>12.2%}")
print(f"{'年化波动率':>15} {std_return:>12.2%} {ew_std_return:>12.2%}")
print(f"{'夏普比率':>15} {sharpe_ratio:>12.2f} {ew_sharpe_ratio:>12.2f}")

# 绘制累积收益对比
plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns, label='TimesFM策略')
plt.plot(ew_cumulative_returns, label='等权重策略')
plt.xlabel('时间')
plt.ylabel('累积收益')
plt.title('策略收益对比')
plt.legend()
plt.grid(True)
plt.show()

# 计算换手率
def calculate_turnover(weights):
    """计算每期的换手率"""
    turnover = np.abs(np.diff(weights, axis=0)).sum(axis=1).mean()
    return turnover

timesfm_turnover = calculate_turnover(test_predictions)
equal_weight_turnover = calculate_turnover(equal_weights)

print("\n换手率分析:")
print(f"TimesFM策略换手率: {timesfm_turnover:.4f}")
print(f"等权重策略换手率: {equal_weight_turnover:.4f}")

# 分析权重集中度
def calculate_herfindahl(weights):
    """计算Herfindahl指数（权重集中度）"""
    return np.mean(np.sum(weights**2, axis=1))

timesfm_concentration = calculate_herfindahl(test_predictions)
equal_weight_concentration = calculate_herfindahl(equal_weights)

print("\n权重集中度分析 (Herfindahl指数):")
print(f"TimesFM策略集中度: {timesfm_concentration:.4f}")
print(f"等权重策略集中度: {equal_weight_concentration:.4f}")